In [1]:
from typing import TypedDict, Literal  #fixed  set of currency

class PortfolioState(TypedDict):      #type hint
    amount_usd: float
    total_usd: float
    target_currency: Literal["INR", "EUR"]
    total: float

In [3]:
def calc_total(state: PortfolioState) -> PortfolioState:
    state['total_usd'] = state['amount_usd'] * 1.08
    return state

def convert_to_inr(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 85
    return state

def convert_to_eur(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 0.9  #hardcoded to 0.9
    return state

def choose_conversion(state: PortfolioState) -> str:
    return state["target_currency"]

In [24]:
#creating my graph

from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node("calc_total_node", calc_total)
builder.add_node("convert_to_inr_node", convert_to_inr)
builder.add_node("convert_to_eur_node", convert_to_eur)

builder.add_edge(START, "calc_total_node")

                                           # conditional age
builder.add_conditional_edges(
    "calc_total_node",
    choose_conversion,
    {
        "INR": "convert_to_inr_node",     #inr goes to inr node
        "EUR": "convert_to_eur_node",      #  eur goes to eur node
    }
)
builder.add_edge(["convert_to_inr_node", "convert_to_eur_node"], END)

graph1 = builder.compile()

In [26]:

graph.invoke({"amount_usd": 1500, "target_currency": "EUR"})

{'amount_usd': 1500,
 'total_usd': 1620.0,
 'target_currency': 'EUR',
 'total': 1458.0}